### Main Loop for the analysis of a list of votables  ...

Features are:
1. Ability to stop and resume loop
2. Ability to plot the results
3. Plug different kinds of analysis.
4. Log results in a file and/or sqlitedb
5. Should be runnable in the downloaded jl file.
6. ETA

In [ ]:
using DataFrames
using CSV
using Statistics

push!(LOAD_PATH,"/home/stephane/Science/cluster/GAIA/master/src")
using GaiaClustering


## directory
rootdir = "/home/stephane/Science/cluster/GAIA"
wdir    = "/home/stephane/Science/cluster/GAIA/products"
votdir  = "/home/stephane/Science/cluster/GAIA/products/votable"
plotdir = "/home/stephane/Science/cluster/GAIA/products/test"

cd(wdir)

In [ ]:
## load a liist of votable and update the file if done
## add results
## 

function readlist_votable(filelist::String)
    vot = CSV.read(filelist)
    return(vot)
end

In [ ]:
function getdata(filevot)
    voname = filevot

    data       = read_votable(voname)
    df         = filter_data(data)
    dfcart     = add_cartesian(df)
    blck       = [[1,2,3],[4,5], [6,7,8]]
    wghtblck   = [4.0,5.0,1.0]
    norm       = "identity"

    dfcartnorm , scale8 = normalization_PerBlock(dfcart, blck, wghtblck , norm, false) 
    return(dfcartnorm , dfcart)
end

In [ ]:
function mcmc_params()
    minQ    = 2.5
    minstars = 50
    ##
    epsmean = 2.0
    epsdisp = 1.5
    min_nei   = 10
    min_cl    = 15
    ncoredisp = 10
    ##
    nburnout  = 500
    niter     = 3000
    pinit = GaiaClustering.abc(minQ, minstars, epsmean,epsdisp,min_nei, min_cl, ncoredisp, nburnout , niter)
    return(pinit)
end

function dbscanmcmc_init(fileres)
    res = []
    found = false
    try
        res = CSV.read(fileres, delim=";")
        found = true
    catch
        println("## No $fileres file, it will be created...")
    end
    return(found)
end

function dbscanmcmc_updt!(initmcmc, fileres, mc ,votname)
    epsm = mean(mc.eps)
    epsd = std(mc.eps)
    mneim = mean(mc.mne)
    mneid = std(mc.mne)
    mclm = mean(mc.mcl)
    mcld = std(mc.mcl)
    qcm = mean(mc.qc)
    qnm = mean(mc.qn)
    qcd = std(mc.qc)
    qnd = std(mc.qn)
    
    println("## DBSCAN/MCMC stats:")
    println("### ϵ : ",epsm," +/- ", epsd)
    println("### min_nei  : ", mneim," +/- ", mneid)
    println("### min_clus  : ", mclm,"+/- ", mcld)
    println("### Qn  : ",qnm," +/- ", qnd) 
    println("### Qc  : ",qcm," +/- ", qcd)
    println("##")
    
    if !initmcmc
        res = DataFrame(votname=votname, epsm = epsm, epsd=epsd, mneim=mneim,mneid=mneid,mclm=mclm,mcld=mcld,
            qcm=qcm,qcd=qcd, qnm=qnm,qnd=qnd)
        CSV.write(fileres,res,delim=';')
        initmcmc = true
    else
        res = CSV.read(fileres, delim=";")
        newrow = DataFrame(votname=votname,epsm = epsm, epsd=epsd, mneim=mneim,mneid=mneid,mclm=mclm,mcld=mcld,
            qcm=qcm,qcd=qcd, qnm=qnm,qnd=qnd)
        println("### add results .....")
        append!(res,newrow)
        CSV.write(fileres,res,delim=';')  
    end
end

function check_mcmc(initmcmc, votname, fileres)
    if !initmcmc
        return(false)
    else
        res = CSV.read(fileres, delim=";")
        if votname in res[:votname]
            return(true)
        else
            return(false)
        end
    end
end

In [ ]:
## Main loop
##

macro main(filelist,fileres)
    let
        println("## Starting main loop ..")
        vot = readlist_votable(filelist)
        println("## $filelist read")
        s=size(vot)
        
        ## check for the mcmc result file...
        mcmcinit  = dbscanmcmc_init(fileres)
        
        for i in 1:s[1]
            votname = vot[:votname][i]
            
            mcmcfound = check_mcmc(mcmcinit, votname, fileres)
            if !mcmcfound
                println("## Starting with $votname")
                dfcartnorm , dfcart = getdata(votdir*"/"*votname)
                params = mcmc_params()
                mc = abc_mcmc_dbscan(dfcartnorm, dfcart, params)
                plot_dbscan_mcmc(plotdir, votname, mc , true)
                dbscanmcmc_updt!(mcmcinit, fileres, mc ,votname)       
            end
            
        end
    end
    print("## Main loop done.")
end

@main("ngclist.csv","ngclistResults.csv")